In [1]:
import pandas as pd 

df = pd.read_csv("HydroOttawaAnnotatedData.csv") 
df.head()

,Tweet,Sentiment
0,"more than 170,000 customers were without power...",0
1,"Hydro Ottawa says it will be days, not hours b...",0
2,"At this time, all available crews have being m...",1
3,It looks like a war scene: Ottawa mayor warns ...,0
4,My thoughts are with all the people in the Ott...,1


In [2]:
pattern_analyzer_sentiment = 'PatternAnalyzerSentiment'
naive_bayes_sentiment = 'NaiveBayesSentiment'
df[pattern_analyzer_sentiment] = 0
df[naive_bayes_sentiment] = 0
df.head()

,Tweet,Sentiment,PatternAnalyzerSentiment,NaiveBayesSentiment
0,"more than 170,000 customers were without power...",0,0,0
1,"Hydro Ottawa says it will be days, not hours b...",0,0,0
2,"At this time, all available crews have being m...",1,0,0
3,It looks like a war scene: Ottawa mayor warns ...,0,0,0
4,My thoughts are with all the people in the Ott...,1,0,0


In [5]:
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer

for index, row in df.iterrows():
    analysis = TextBlob(row['Tweet'])
    if analysis.sentiment.polarity > 0:     # Positive
        df.loc[index, pattern_analyzer_sentiment] = 1
    elif analysis.sentiment.polarity <= 0:   # Negative
        df.loc[index, pattern_analyzer_sentiment] = 0

nba = NaiveBayesAnalyzer()
for index, row in df.iterrows():
    analysis = TextBlob(row['Tweet'], analyzer=nba)
    if analysis.sentiment.classification == 'pos':
        df.loc[index, naive_bayes_sentiment] = 1
    elif analysis.sentiment.classification == 'neg':
        df.loc[index, naive_bayes_sentiment] = 0

df.PatternAnalyzerSentiment = df.PatternAnalyzerSentiment.astype(int)
df.NaiveBayesSentiment = df.NaiveBayesSentiment.astype(int)
df.head()

,Tweet,Sentiment,PatternAnalyzerSentiment,NaiveBayesSentiment
0,"more than 170,000 customers were without power...",0,1,0
1,"Hydro Ottawa says it will be days, not hours b...",0,0,1
2,"At this time, all available crews have being m...",1,1,1
3,It looks like a war scene: Ottawa mayor warns ...,0,0,1
4,My thoughts are with all the people in the Ott...,1,1,0


In [8]:

def get_accuracy(pattern_type):
    num_accurate = 0
    for index, row in df.iterrows():
        if row['Sentiment'] == row[pattern_type]:
            num_accurate += 1
    return (num_accurate/len(df.index))*100

print('Accuracy with PatternAnalyzer: ', get_accuracy(pattern_analyzer_sentiment), '%')
print('Accuracy with NaiveBayesAnalyzer: ', get_accuracy(naive_bayes_sentiment), '%')

Accuracy with PatternAnalyzer:  59.66386554621849 %
Accuracy with NaiveBayesAnalyzer:  60.08403361344538 %


In [9]:
df.to_csv('hydro_ottawa_naive_pattern.csv', index=False)